# Confusion matrix for CocoAPI

The "main_Evaluate_Object_Detection.ipynb" uses the CocoApi to evaluate the model performance, e.g., AP50. But that code does not include the output of confusion matrix (e.g., TP, FP, and FN).

This script provides the output of confusion matrix using **FiftyOne** tool

FiftyOne’s implementation of COCO-style evaluation matches the reference implementation available via pycocotools (https://github.com/cocodataset/cocoapi).

Refer to: 
https://docs.voxel51.com/integrations/coco.html#loading-coco-formatted-data

## Install fiftyone and cocoapi

In [ ]:
#   For windows:
#        pip3 install fiftyone --user

#   For windows:
#        install pycocotools-2.0:
#        pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI

## Load coco dataset (ground-truth)

**Note**: 

(1) the folder/path of images in json file must contains " / " instead of " \ " 

(2) In json file, "image_id" and the "id" for annotations must not start "0", otherwise the results will be not correct

In [ ]:
import fiftyone as fo

# A name for the dataset
# name = "my-dataset"

# GJO test dataset
image_path = r"U:/AIMMW/Tianlong/Peng/Paper_SSL/Dataset/labeled_data/subsets/Test/test"
labels_path = r"U:/AIMMW/Tianlong/Peng/Paper_SSL/Dataset/labeled_data/subsets/Test/test.json"

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset  # coco format

dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=image_path,
    labels_path=labels_path,
    # include_id=True,
    # name=name,
)
# print(dataset)

In [ ]:
# show the first data in the dataset,
# Carefully check the "label" name!
# Note the coordinates of the bbox is scaled into [0,1]

# print(dataset.first().detections.detections[0])

## Load predictions

When inferencing a model on a test dataset using Detectron2, it will output a prediction file named "coco_instances_results.json" with predicted bbox

In [ ]:
import fiftyone.utils.coco as fouc
import json

#
# Mock COCO predictions, where:
# - `image_id` corresponds to the `coco_id` field of `coco_dataset`
# - `category_id` corresponds to classes in `coco_dataset.default_classes`
#
# predictions = [
#     {"image_id": 1, "category_id": 18, "bbox": [258, 41, 348, 243], "score": 0.87},
#     {"image_id": 2, "category_id": 11, "bbox": [61, 22, 504, 609], "score": 0.95},
# ]


with open(r'U:\AIMMW\Tianlong\Peng\Paper_SSL\Results\Predicted\SC_FTAL_F2_80%_best\coco_instances_results.json', 'r') as fcc_file:
    fcc_data = json.load(fcc_file)
# print(fcc_data)
    
predictions = fcc_data


# Add COCO predictions to `predictions` field of dataset
classes = ['hold_space', 'entrap bean', 'free bean',]
# # classes = ['hold_space', 'litter',]
fouc.add_coco_labels(dataset, "predictions", predictions, classes)

# # Verify that predictions were added to two images
print(dataset.count("predictions"))  # 2

In [ ]:
dataset

In [ ]:
# show the first data (with prediction bbox) in the dataset ,
# Carefully check the "label" name!
# Note the coordinates of the predicted bbox is scaled into [0,1]

# print(dataset.first())

In [ ]:
# sample = dataset.first()
# print(sample.detections.detections[0])

## Output results 

by comparing ground-truth labels and predictions bbox

**Note**: Predicted and ground truth objects are matched using a specified IoU threshold (default = 0.50). This threshold can be customized via the iou parameter. The IoU selection affects the calculation of TP, FN, are FP value.

In [9]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Evaluate the objects in the `predictions` field with respect to the
# objects in the `ground_truth` field
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    iou=0.5,
    method="coco",   
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |███████████████████| 29/29 [114.9ms elapsed, 0s remaining, 252.4 samples/s]     
Performing IoU sweep...
 100% |███████████████████| 29/29 [90.6ms elapsed, 0s remaining, 320.1 samples/s] 


### mAP

Note: this is the mAP at IoU=.50:.05:.95

In [10]:
print(results.mAP())

0.0


### TP, FP, FN

In [11]:
# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

TP: 0
FP: 18
FN: 63


### Precision-recall curve

In [ ]:
plot = results.plot_pr_curves(classes=["litter"])
plot.show()

### Confusion matrix

In [12]:
import fiftyone as fo
import fiftyone.zoo as foz


# Perform evaluation, allowing objects to be matched between classes
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    method="coco",
    classwise=True,
)

# Generate a confusion matrix for the specified classes
plot = results.plot_confusion_matrix(classes = ['hold_space', 'entrap bean', 'free bean'])
plot.show()

Evaluating detections...
 100% |███████████████████| 29/29 [83.8ms elapsed, 0s remaining, 346.1 samples/s] 


FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': 'a18f274c-5e70-4404-ae8e-b7aa89b2d96d',
              'x': array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]),
              'y': array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '6480b236-3071-4f2b-88a2-d2920fb7093b',
              'z': array([[ 0,  4, 14,  0],
                          [ 0,  0,  0, 39],
                          [ 0,  0,  